# RiskPrediction_Dataset.csv 데이터를 사용하여 Kafka를 통해 실시간 데이터 스트리밍을 구현
***

**목차**
1. Kafka란 무엇인가?
2. Kafka를 사용한 데이터 스트리밍 개요
3. Kafka 설치 및 설정
    - Windows 환경에서 Kafka 설치
4. Kafka 프로듀서와 컨슈커 구현
    - 프로듀서 코드 작성
    - 컨슈머 코드 작성
5. RiskPrediction_Dataset.csv 데이터를 Kafka로 스트리밍
    - 데이터 파일을 읽어 프로듀서로 전송
6. Kafka와 기존 파이프라인 통합
7. 추가 고려 사항 및 제안
8. 마무리

***
## **1. Kafka란 무엇인가?**

### **Apache Kafka 소개**

**Apache Kafka**는 오픈 소스 분산 이벤트 스트리밍 플랫폼으로, 대용량의 실시간 데이터를 효율적으로 처리하고 저장할 수 있습니다. Kafka는 다음과 같은 기능을 제공합니다.
- **발행/구독 메시징 시스템**: 프로듀서(데이터 생산자)가 보낸 메시지를 토픽(topic)에 저장하고, 컨슈머(데이터 소비자)가 해당 토픽을 구독하여 메시지를 받아갑니다.
- **높은 처리량과 확장성**: 분산 시스템으로 설계되어 대량의 데이터를 빠르게 처리하고, 필요에 따라 클러스터를 확장할 수 있습니다.
- **내구성**: 메시지를 디스크에 저장하여 데이터 손실을 방지합니다.
- **스트리밍 데이터 처리**: 실시간 데이터 스트림을 처리하고, 데이터 파이프라인을 구축하는데 사용됩니다.

### **Kafka의 주요 구성 요소**

- **토픽(Topic)**: 메시지가 저장되는 카테고리 또는 피드 이름입니다.
- **프로듀서(Producer)**: 토픽에 데이터를 게시하는 클라이언트입니다.
- **컨슈머(Consumer)**: 토픽으로부터 데이터를 구독하여 가져가는 클라이언트입니다.
- **브로커(Broker)**: Kafka 서버로, 토픽과 메시지를 관리합니다.
- **파티션(Partition)**: 토픽은 하나 이상의 파티션으로 분할되어 데이터 병렬 처리를 가능하게 합니다.

***

## **2. Kafka를 사용한 데이터 스트리밍 개요**

> Kafka를 사용하면 **RiskPrediction_Dataset.csv**와 같은 대용량의 시계열 데이터를 실시간으로 처리할 수 있습니다. 데이터 파이프라인은 다음과 같이 구성

1. **프로듀서(Producer)**: CSV 파일에서 데이터를 읽어 Kafka 토픽에 메시지로 전송합니다.
2. **Kafka 브로커(Broker)**: 메시지를 토픽에 저장하고 관리합니다.
3. **컨슈머(Consumer)**: 토픽으로부터 메시지를 읽어와 실시간 데이터 처리 및 분석을 수행합니다.

***

## **3. Kafka 설치 및 설정**


### **Windows 환경에서 Kafka 설치**

> Kafka는 Java로 작성되었기 때문에 Java 런타임 환경(JRE)이 필요합니다.

**a. Java 설치**   

1. **Java SE Development Kit (JDK)** 다운로드 및 설치:   
    - [Oracle 공식 사이트](https://www.oracle.com/java/technologies/downloads/#java17?er=221886)에서 최신 JDK를 다운로드합니다.   
    - 설치 후 환경 변수를 설정하여 `java` 명령어를 사용할 수 있도록 합니다.
        
**b. Kafka 다운로드**   

1. **Kafka 다운로드**:   
    - [Apache Kafka 공식 웹사이트](https://kafka.apache.org/downloads)에서 최신 Kafka 바이너리를 다운로드합니다.   
    - 예를 들어, **kafka_2.13-3.4.0** 버전을 다운로드합니다.   
2. **압축 해제**:
    - 다운로드한 **.tgz** 파일을 압축 해제하고, 원하는 디렉토리에 저장합니다.   
    - 예를 들어, `C:\kafka` 디렉토리에 Kafka를 설치합니다.   
        
**c. Zookeeper 및 Kafka 브로커 실행**   
> Kafka는 Zookeeper에 의존하므로, Zookeeper를 먼저 실행해야 합니다.   

1. **Zookeeper 실행 스크립트 작성**   
    - `C:\kafka\config\zookeeper.properties` 파일을 사용하여 Zookeeper를 실행합니다.   
    - 명령 프롬프트를 열고 다음 명령어를 실행합니다:
      ```
      cd C:\kafka
      .\bin\windows\zookeeper-server-start.bat .\config\zookeeper.properties
      ```
    - Zookeeper가 성공적으로 시작되면 다른 명령 프롬프트를 열어 Kafka 브로커를 실행합니다.

2. **Kafka 브로커 실행**
    - Kafka 브로커 설정 파일은 `C:\kafka\config\server.properties` 입니다.
    - 새로운 명령 프롬프트에서 다음 명령어를 실행합니다:
      ```
      cd C:\kafka
      .\bin\windows\kafka-server-start.bat .\config\server.properties
      ```
    - Kafka 브로커가 성공적으로 실행되면 Kafka를 사용할 준비가 완료됩니다.

***

## **4. Kafka 프로듀서와 컨슈머 구현**

> Kafka를 사용하여 데이터를 전송하고 수신하기 위해 프로듀서와 컨슈머를 Python으로 구현함.   
> 이를 위해 **kafka-python** 라이브러리를 사용함.

### **a. kafka-python 라이브러리 설치**

    ```
    pip install kafka-python
    ```

### **b. 프로듀서 코드 작성**

> CSV 파일에서 데이터를 읽어 Kafka 토픽에 메시지로 전송하는 프로듀서 코드를 작성함.

### **c. 컨슈머 코드 작성**

> Kafka 토픽에서 메시지를 읽어와 실시간 데이터 처리를 수행하는 컨슈머 코드를 작성함.

In [ ]:
# 